Instalação das dependências e preparo do ambiente.

In [ ]:
!pip install -q transformers accelerate bitsandbytes torch

Caso receba erro de import das bibliotecas, é só atualizar:

In [ ]:
!pip install -U -q transformers accelerate bitsandbytes

Primeiro, precisamos entender se é possível enviar um hidden_state diretamente como input de um modelo.
Premissas:
1. Os dois modelos devem ser exatamente iguais, para que tenhamos outputs compatíveis entre eles.
2. Foi necessário quantizar os modelos para rodar localmente no Colab.
3. Foi necessário converter as matrizes de pensamento todas para o tipo de saída do modelo (bfloat16) para garantir compatibilidade.

In [ ]:
import gc
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Usando uma versão acessível do Llama 3 Instruct para não lidar com autenticação exigida pelo modelo oficial
model_id = "NousResearch/Meta-Llama-3-8B-Instruct"

print(f"Carregando {model_id} (Isso pode demorar um pouco)...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("Llama 3 Carregado. Preparando injeção V2V...")
print("-" * 50)

# --- PREPARAÇÃO DO PENSAMENTO (Agente Emissor) ---

messages = [
    {"role": "user", "content": "Escreva uma função Python eficiente que receba uma lista e retorne apenas os números pares. Apenas o código."}
]
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

with torch.no_grad():
    # Extração do Vetor
    vector_thought = model.get_input_embeddings()(input_ids)

    print(f"1. Instrução Vetorial Criada. Shape: {vector_thought.shape}")

    # --- O EXPERIMENTO V2V (Agente Receptor) ---
    print("2. Injetando pensamento no Agente Receptor...")

    generated_ids = model.generate(
        inputs_embeds=vector_thought, # O modelo recebe o vetor, não o texto.
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1
    )

    output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("-" * 50)
print("RESPOSTA DO AGENTE (Gerada via Vetor):")
print(output_text)

A esse ponto, descobri que, sim, **é possível** enviar hidden states diretamente como inputs para modelos de linguagem, sem a necessidade da última etapa de serialização para texto.

> RESPOSTA DO AGENTE (Gerada via Vetor):

```
> def pares(lista):
return [i for i in lista if i % 2 == 0]
```













In [ ]:
# Fase 2: Capturar e enviar o embedding antes que o modelo o transforme em texto.
# Objetivo: Agente A processa uma info e passa o contexto para o Agente B continuar.

print("Iniciando Fase 2: Transferência de Estado Oculto (Hidden State)...")

# 1. O AGENTE "PLANEJADOR" (Vector)
prompt_agente_a = [
    {"role": "user", "content": "Analise o seguinte problema: Criar uma API de login. Liste os 3 passos técnicos principais. Seja breve."}
]
input_ids_a = tokenizer.apply_chat_template(prompt_agente_a, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs_a = model(input_ids_a, output_hidden_states=True)

    # Pegamos o último estado oculto .
    # Shape: [Batch, Sequence Length, Hidden Size] (Ex: [1, 45, 4096])
    last_hidden_state_a = outputs_a.hidden_states[-1]

    print(f"Agente A 'pensou'. Tamanho do pensamento: {last_hidden_state_a.shape}")

    # Num sistema real, enviaríamos esse tensor pela rede/memória compartilhada.
    # Aqui, vamos passar direto para o Agente B.

    # --- O AGENTE "CODER" (Receptor) ---
    # O Agente B recebe o pensamento do A e deve continuar a partir dali.
    # Ele não deve recer o input original em string. Ele vê apenas o output embedded do A.

    # Projetamos o hidden_state de volta para o espaço de embedding ou usamos como contexto?
    # Para o primeiro teste, vamos tentar usar o hidden state como input_embeds.
    # *Nota Teórica:* Isso geralmente requer um adaptador, mas vamos testar a robustez do Llama 3 "cru".

    print("Injetando raciocínio do Agente A no Agente B...")

    generation_b = model.generate(
        inputs_embeds=last_hidden_state_a,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7
    )

    output_text_b = tokenizer.decode(generation_b[0], skip_special_tokens=True)

print("-" * 50)
print("O QUE O AGENTE B GEROU:")
print(output_text_b)

Como esperado, não seria tão simples. É necessário usar um normalizador para que o output esteja no mesmo "formato" do input esperado pelo Llama.

*Output:*

```
O QUE O AGENTE B GEROU:
 toalalsicalalalischalichtalal...

ichalaln duplex:alichtalific autor...

ich, atried to create to be anal of simple technique to carry the firsts, carrying,alisch to: the firstln'tal, return,alischal to the firstalitzals of the last, a dearzichals for treatment at the time. (ried:ried, and return toisch to the brain, withou: a virtual, and

```



In [ ]:
patience = 5
patience_counter = 0
best_loss = float('inf')
min_delta = 0.0010

training_data = [
    {"instruction": "Escreva uma função soma em Python.", "target_code": "def soma(a, b): return a + b"},
    {"instruction": "Crie uma lista de números pares até 10.", "target_code": "pares = [x for x in range(11) if x % 2 == 0]"},
    {"instruction": "Imprima 'Ola Mundo' em Python.", "target_code": "print('Ola Mundo')"},
    {"instruction": "Função para multiplicar dois valores.", "target_code": "def mult(x, y): return x * y"}
]

# Adaptador
class VectorAdapter(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(VectorAdapter, self).__init__()
        self.projector = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x_float = x.to(torch.float32)
        out = self.projector(x_float)
        return out.to(torch.float16)

adapter = VectorAdapter(4096, 4096).to("cuda")
optimizer = optim.AdamW(adapter.parameters(), lr=5e-4)
loss_fn = nn.CrossEntropyLoss()

print(f"Iniciando Treino (Paciência: {patience})...")
print("-" * 50)

epochs = 200
loss_history = []

model.eval()

for epoch in range(epochs):
    epoch_loss = 0

    for item in training_data:
        optimizer.zero_grad()

        # Input do Agente A
        messages_a = [{"role": "user", "content": item['instruction']}]
        input_a = tokenizer.apply_chat_template(messages_a, return_tensors="pt").to("cuda")
        target_tokens = tokenizer(item['target_code'], return_tensors="pt", add_special_tokens=False).to("cuda").input_ids

        with torch.no_grad():
            outputs_a = model(input_a, output_hidden_states=True)
            thought_vector = outputs_a.hidden_states[-1][:, -1, :]
            target_embeddings = model.get_input_embeddings()(target_tokens)

        # Para acelerar o teste das hipóteses, estamos tentando adivinhar somente o primeiro token da resposta.
        # Por isso, concatenamos com o restante da resposta desejada, e vamos medir a acurácia em cima disso.
        adapted_vector = adapter(thought_vector).unsqueeze(1)
        inputs_seq = torch.cat([adapted_vector, target_embeddings[:, :-1, :]], dim=1)

        outputs_b = model(inputs_embeds=inputs_seq)
        logits = outputs_b.logits
        shift_logits = logits.view(-1, model.config.vocab_size)
        shift_labels = target_tokens.view(-1)
        loss = loss_fn(shift_logits, shift_labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(adapter.parameters(), 1.0)
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(training_data)

    if avg_loss < (best_loss - min_delta):
        best_loss = avg_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if (epoch+1) % 10 == 0:
        print(f"Época {epoch+1}/{epochs} - Loss: {avg_loss:.4f} - Paciência: {patience_counter}/{patience}")

    if patience_counter >= patience:
        print(f"\n✋ PARADA ANTECIPADA na época {epoch+1}.")
        print(f"Melhor Loss alcançada: {best_loss:.4f}")
        break


print("-" * 50)
test_instruction = "Escreva uma função soma em Python."
print(f"Testando instrução: '{test_instruction}'")
msg_test = [{"role": "user", "content": test_instruction}]
input_test = tokenizer.apply_chat_template(msg_test, return_tensors="pt").to("cuda")

with torch.no_grad():
    out_test = model(input_test, output_hidden_states=True)
    thought_test = out_test.hidden_states[-1][:, -1, :]
    projected_thought = adapter(thought_test).unsqueeze(1)
    gen = model.generate(
        inputs_embeds=projected_thought,
        max_new_tokens=30,
        do_sample=False,
        repetition_penalty=1.2
    )
    res = tokenizer.decode(gen[0], skip_special_tokens=True)

print(f"Resposta V2V: {res}")

A intenção do primeiro token (devido nosso dataset ser extremamente simples) ainda é muito forte, então alucinamos.

Em vez de usar uma regressão linear, vamos construir adicionar uma camada de rede neural pra capturar o restante do contexto.

In [ ]:
torch.cuda.empty_cache()
gc.collect()
print("Limpeza de VRAM.")

# Congelamos os pesos pra não estourar a VRAM.
for param in model.parameters():
    param.requires_grad = False

training_data = [
    {"instruction": "Escreva uma função soma em Python.", "target_code": "def soma(a, b): return a + b"},
    {"instruction": "Crie uma lista de números pares até 10.", "target_code": "pares = [x for x in range(11) if x % 2 == 0]"},
    {"instruction": "Imprima 'Ola Mundo' em Python.", "target_code": "print('Ola Mundo')"},
    {"instruction": "Função para multiplicar dois valores.", "target_code": "def mult(x, y): return x * y"}
]

class MLPAdapter(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLPAdapter, self).__init__()

        # Reduzimos a camada oculta para 1024 (Economiza vram e ainda é não-linear)
        hidden_dim = 1024

        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x_float = x.to(torch.float32)
        out = self.net(x_float)
        return out.to(torch.float16)

adapter = MLPAdapter(4096, 4096).to("cuda")
optimizer = optim.AdamW(adapter.parameters(), lr=2e-4, weight_decay=0.01)
loss_fn = nn.CrossEntropyLoss()


def evaluate_latent_alignment(model, adapter, test_data, tokenizer):
    """
    Avalia o quão próximo o vetor projetado pelo Agente A está
    do vetor de embedding real do Agente B (Target).

    Retorna: Similaridade de Cosseno Média (-1 a 1).
    Quanto mais próximo de 1, melhor o alinhamento semântico.
    """
    model.eval()
    adapter.eval()

    similarities = []

    print("\n--- 📐 Avaliação de Geometria Latente ---")

    with torch.no_grad():
        for item in test_data:
            instruction = item['instruction']
            target_code = item['target_code']

            msg_a = [{"role": "user", "content": instruction}]
            input_ids_a = tokenizer.apply_chat_template(msg_a, return_tensors="pt").to("cuda")

            outputs_a = model(input_ids_a, output_hidden_states=True)
            thought_vector_a = outputs_a.hidden_states[-1][:, -1, :]
            projected_vector = adapter(thought_vector_a)

            target_tokens = tokenizer(target_code, return_tensors="pt", add_special_tokens=False).to("cuda").input_ids
            first_token_target = target_tokens[:, 0].unsqueeze(0)
            target_embedding_b = model.get_input_embeddings()(first_token_target).squeeze(1)

            cos_sim = F.cosine_similarity(projected_vector, target_embedding_b, dim=-1)
            similarities.append(cos_sim.item())

            print(f"Instrução: {instruction[:30]}... | Sim: {cos_sim.item():.4f}")

    avg_sim = sum(similarities) / len(similarities)
    print(f"📊 Similaridade Média do Espaço Latente: {avg_sim:.4f}")
    print("-----------------------------------------\n")

    return avg_sim


print("Adaptador MLP minimizado inicializado.")
print("Iniciando Treino...")
print("-" * 50)

epochs = 150
loss_history = []

model.eval()
aux_loss_fn = nn.MSELoss()
lambda_align = 10.0

print("Iniciando Treino com Loss Híbrida (Texto + Geometria)...")
print("-" * 50)

for epoch in range(epochs):
    epoch_loss = 0
    epoch_align_loss = 0
    adapter.train()

    for item in training_data:
        optimizer.zero_grad()

        with torch.no_grad():
            messages_a = [{"role": "user", "content": item['instruction']}]
            input_a = tokenizer.apply_chat_template(messages_a, return_tensors="pt").to("cuda")
            target_tokens = tokenizer(item['target_code'], return_tensors="pt", add_special_tokens=False).to("cuda").input_ids

            outputs_a = model(input_a, output_hidden_states=True)
            thought_vector = outputs_a.hidden_states[-1][:, -1, :]

            # Queremos que o output do adaptador se pareça com o embedding do 1º token da resposta
            first_token_target = target_tokens[:, 0].unsqueeze(0)
            target_embedding_b = model.get_input_embeddings()(first_token_target).squeeze(1) # [1, 4096]

            # Para o texto, continuamos precisando da sequência completa
            target_embeddings_seq = model.get_input_embeddings()(target_tokens)

        # O adaptador gera o vetor
        adapted_vector = adapter(thought_vector)

        # Forçamos 'adapted_vector' a ser igual a 'target_embedding_b'
        loss_align = aux_loss_fn(adapted_vector.float(), target_embedding_b.float())

        # Continua o fluxo para o decoder (Loss de Texto)
        adapted_vector_seq = adapted_vector.unsqueeze(1)
        inputs_seq = torch.cat([adapted_vector_seq, target_embeddings_seq[:, :-1, :]], dim=1)

        outputs_b = model(inputs_embeds=inputs_seq)
        logits = outputs_b.logits
        shift_logits = logits.view(-1, model.config.vocab_size)
        shift_labels = target_tokens.view(-1)

        loss_text = loss_fn(shift_logits, shift_labels)
        total_loss = loss_text + (lambda_align * loss_align)

        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(adapter.parameters(), 1.0)
        optimizer.step()

        epoch_loss += total_loss.item()
        epoch_align_loss += loss_align.item()

if (epoch + 1) % 20 == 0:
        evaluate_latent_alignment(model, adapter, training_data, tokenizer)
        mse_medio = epoch_align_loss / len(training_data)
        print(f"🔍 Diagnóstico de Loss:")
        print(f"   - Loss Geometria (MSE): {mse_medio:.6f} (Peso: {lambda_align})")
        print(f"   - Loss Total (Híbrida): {avg_loss:.4f}")


print("-" * 50)
test_instruction = "Escreva uma função soma em Python."
print(f"Testando: '{test_instruction}'")

msg_test = [{"role": "user", "content": test_instruction}]
input_test = tokenizer.apply_chat_template(msg_test, return_tensors="pt").to("cuda")

with torch.no_grad():
    out_test = model(input_test, output_hidden_states=True)
    thought_test = out_test.hidden_states[-1][:, -1, :]
    projected_thought = adapter(thought_test).unsqueeze(1)
    gen = model.generate(
        inputs_embeds=projected_thought,
        max_new_tokens=40,
        do_sample=False,
        repetition_penalty=1.2
    )
    res = tokenizer.decode(gen[0], skip_special_tokens=True)

print(f"Resposta do nosso V2V: {res}")

Estamos rodando em "modo econômico", então é praticamente esperado que ele entre em loop.

In [ ]:
# --- TESTE DE RESGATE (Inferência Calibrada) ---
print("Testando com parâmetros Anti-Loop Agressivos...")

test_instruction = "Escreva uma função soma em Python."
msg_test = [{"role": "user", "content": test_instruction}]
input_test = tokenizer.apply_chat_template(msg_test, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs_a = model(input_test, output_hidden_states=True)
    thought_test = outputs_a.hidden_states[-1][:, -1, :]

    projected_thought = adapter(thought_test).unsqueeze(1)

    print(f"Instrução: '{test_instruction}'")
    print("Gerando...")

    gen = model.generate(
        inputs_embeds=projected_thought,
        max_new_tokens=50,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        top_k=50,
        repetition_penalty=2.5,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id
    )

    res = tokenizer.decode(gen[0], skip_special_tokens=True)

print("-" * 50)
print(f"Nova resposta V2V: {res}")

Conseguimos fugir do loop do primeiro token "defdefdef", pelo menos.

In [ ]:
print("Teste de Direção Semântica...")
instrucao_soma = "Escreva uma função soma em Python."
instrucao_mult = "Função para multiplicar dois valores."


for inst in [instrucao_soma, instrucao_mult]:
    msg = [{"role": "user", "content": inst}]
    inp = tokenizer.apply_chat_template(msg, return_tensors="pt").to("cuda")

    with torch.no_grad():
        thought = model(inp, output_hidden_states=True).hidden_states[-1][:, -1, :]
        vec = adapter(thought).unsqueeze(1)

        # O objetivo aqui é validar se ele recebeu a intenção do prompt inicial,
        # ou se só tá escrevendo def "aleatoriamente"; então fazemos o contrário do que fizemos lá em cima:
        # Passamos o def e deixamos ele completar o resto do código.
        start_token = tokenizer("def ", return_tensors="pt").to("cuda").input_ids

        # Input = [vetor] + ["def"]
        inputs_seq = torch.cat([vec, model.get_input_embeddings()(start_token)], dim=1)
        gen = model.generate(
            inputs_embeds=inputs_seq,
            max_new_tokens=20,
            do_sample=False
        )
        res = tokenizer.decode(gen[0], skip_special_tokens=True)

    print(f"\nInstrução: {inst}")
    print(f"Resultado: {res}")

Para essa etapa do estudo, podemos considerar um **sucesso**:
1. O modelo entendeu que deve gerar código Python; mas ainda não capturou a intenção do código (somar vs multiplicação).

Nosso próximo passo é balancear os pesos e fazer com que o modelo consiga diferenciar os prompts simples.
Depois disso, vamos elevando a complexidade dos prompts (que vai precisar de mais recursos de hardware pra funcionar; vamos ter que diminuir as quantizações e diminuições que fizemos).

# O Resultado?


> **Para criarmos sistemas Multi-Agentes de LLM, não precisamos necessariamente da camada de serialização para texto intermediária.**


É *possível* criar um protocolo de comunicação que envie intenções latentes entre os modelos/agentes ao invés de texto puro.
Mas **precisamos** construir um dataset de treino efetivo para validar a generalização e persistência semântica.